In [ ]:
if True:
    from julia.api import Julia
    jl = Julia(compiled_modules=False)

#import julia; julia.install(quiet=True)
from julia import Main

import numpy     as np
import panel     as pn; pn.extension()
import holoviews as hv; hv.extension( "bokeh", logo=False)

from panel.interact import interact

def raster(img):  return hv.Image(img).opts(cmap="gray", xaxis=None, yaxis=None, frame_width=200, aspect='equal')

In [ ]:
%load_ext julia.magic

In [3]:
%%julia
using LinearAlgebra;

<div style="height:2cm;">
<div style="float:center;width:100%;text-align:center;"><strong style="height:100px;color:darkred;font-size:40px;">Applications of Subspace Distances</strong>
</div></div>

Subspace distances appear in a wide range of applications.

Here we apply the tools from previous notebooks to PCA drift detection, subspace clustering, and signal analysis.


## Applications of Subspace Distances

Principal angle–based distances appear in many tasks  
where data or signals lie in low-dimensional subspaces.

We apply the metrics introduced earlier to three settings:  
PCA drift detection, anomaly detection, and subspace clustering.

---

### PCA Drift Detection

Given a streaming data source, monitor the leading principal subspace.  
Compare $U_t$ and $U_{t-1}$ using:

- Spectral distance → detects large changes  
- Frobenius distance → smooth sensitivity to gradual drift

Visualize angle trajectory and metric response over time.

---

### Anomaly Detection via Subspace Change

Anomalies can shift the data subspace.  
Track deviation of current subspace $U_t$ from reference $U_0$.

Use:

- Projection distance → sensitive to operator changes  
- Chordal distance → robust to small fluctuations

---

### Subspace Clustering

Data points lie near multiple subspaces.  
Cluster them by pairwise subspace distance.

- Build affinity matrix from projection or geodesic distances  
- Apply spectral clustering  
- Compare results across distance metrics

---

### Metric Comparison

Each distance captures a different notion of subspace difference:

- Spectral: peak deviation  
- Frobenius: total energy  
- Projection: matrix/operator change  
- Geodesic: intrinsic geometry

Choose the metric that matches what matters in your task.

---

### Summary

Subspace distances are not interchangeable.  
Different applications favor different metrics — depending on  
sensitivity to local variation, noise, or geometric structure.

